In [3]:
from DataLoader import DataLoader
from Player import Player
from CollectiveStats import CollectiveStats
from tools import save_json, open_json, compare_dates
import time
import requests
from pprint import pprint

In [ ]:
#this gets all of the seasons we can scrape


url = "https://api-nba-v1.p.rapidapi.com/seasons"

headers = {
	"x-rapidapi-key": "6d8471b900msh64f82e14a3ea8c4p1e6eedjsn363222b8f5fe",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())

In [ ]:
#https://rapidapi.com/api-sports/api/api-nba/playground/apiendpoint_5b1e1e9a-1497-45b5-b9fb-50f9212d4421

#player statistics per game id




url = "https://api-nba-v1.p.rapidapi.com/players/statistics"

querystring = {"game":"8133"}

headers = {
	"x-rapidapi-key": "6d8471b900msh64f82e14a3ea8c4p1e6eedjsn363222b8f5fe",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())


In [ ]:
#https://rapidapi.com/api-sports/api/api-nba/playground/apiendpoint_be16eeb7-56ae-4b3b-93c8-a6392c29626e

#games per season





def games_per_season(season: int) -> dict:
    season = str(season)
    url = "https://api-nba-v1.p.rapidapi.com/games"

    querystring = {"season":season}

    headers = {
        "x-rapidapi-key": "6d8471b900msh64f82e14a3ea8c4p1e6eedjsn363222b8f5fe",
        "x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()


seasons = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]





for szn in seasons:
    games = games_per_season(szn)
    save_json('seasons/'+str(szn)+'_season.json', games)

In [ ]:
games = games_per_season(2024)
save_json('seasons/'+str(szn)+'_season.json', games)

Let's just analyze a specific game's file size to see how big each individual game is. I think we'll have between 13k-14k games.

In [ ]:
def stats_per_gameid(game_id: str) -> dict:
    url = "https://api-nba-v1.p.rapidapi.com/players/statistics"

    querystring = {"game":game_id}

    headers = {
        "x-rapidapi-key": "6d8471b900msh64f82e14a3ea8c4p1e6eedjsn363222b8f5fe",
        "x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()


game = stats_per_gameid('2842')

save_json('test_game.json',game)

In [ ]:


game_ids = {}


for szn in seasons:
    season = open_json('seasons/'+str(szn)+'_season.json')
    print(season.keys())
    season = season['response']
    cur = []
    for i in range(len(season)):
        cur.append(season[i]['id'])
    game_ids[szn] = cur


    

In [ ]:
print(len(game_ids))

arr = [i+1 for i in range(14367)]



In [ ]:
gaps = []
for i in range(1, len(game_ids)):
    if game_ids[i]-game_ids[i-1] > 1: gaps.append(game_ids[i]-game_ids[i-1])

print(gaps)

why are there gaps in the ids? that's odd. whatever           

In [ ]:


for season, games in game_ids.items():
    season_dict = {}
    for idx, game_id in enumerate(games):
        game = stats_per_gameid(game_id)
        if 'response' not in game.keys():
            print(game)
            raise Exception('BAD RESPONSE!')
        season_dict[game_id] = game
        time.sleep(0.14)
        done = (idx+1) / len(games) * 100
        if idx % 100 == 0:
            print(str(done) + '% done! (season '+ str(season)+')')

        

    save_json('games/all_games_'+str(season)+'.json',season_dict)
    

In [ ]:
games = open_json('games/all_games_2015.json')
print(games.keys())

In [ ]:
temp = games['1']['response']
for item in temp:
    print(item)

In [ ]:
import requests
for i in range(30):
    url = "https://api-nba-v1.p.rapidapi.com/players"

    querystring = {"team":i+1,"season":"2021"}

    headers = {
        "x-rapidapi-key": "6d8471b900msh64f82e14a3ea8c4p1e6eedjsn363222b8f5fe",
        "x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())

In [ ]:
seasons = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
players = {}

#if compare_date(game date to this date): this game is a playoff game if True and regular season if False
playoff_dates = ['2016-04-13','2017-04-12','2018-04-11','2019-04-10','2020-08-14','2021-05-16','2022-04-10','2023-04-09','2024-04-14','2025-04-13']

critical_dates = {seasons[i]: playoff_dates[i] for i in range(len(seasons))}


for year in seasons:
    all_games = open_json('games/all_games_'+str(year)+'.json')
    season = open_json('seasons/'+str(year)+'_season.json')['response']
    game_dates = {str(game['id']): game['date']['start'] for game in season}
    
    for game_id, game in all_games.items():
        players_stats = game['response']

        #if date is playoff game or nah
        regular_season = not compare_dates(game_dates[game_id], critical_dates[year])
            
        for p in players_stats:
            if p['player']['id'] is None: continue
            if p['points'] is None: continue
            if p['comment'] and ('DNP' in p['comment'] or 'DND' in p['comment']): continue
            player_id, full_name = p['player']['id'], str(p['player']['firstname'] + ' ' + p['player']['lastname'])

            temp_player = Player(player_id, full_name) if not player_id in players else players[player_id] #create or fetch player
            temp_player.add_game(year, game_id, regular_season, p)
            players[player_id] = temp_player






# POPULATE PLAYER OBJECTS

In [ ]:
players_dict = {player_id: player.to_dict() for player_id, player in players.items()}

save_json('all_players.json', players_dict)

In [ ]:

data = DataLoader()

data.load_player_stats('all_players.json')




In [ ]:
#data.players['10']
data.players['265'].populate_calculated_stats(from_scratch = True)


In [ ]:
print(len(data.players['265'].basic_stats['2024']['regular_season']))
games_list = data.players['265'].basic_stats['2024']['regular_season'].values()
data.players['265'].pretty_print()

# POPULATE COLLECTIVESTATS OBJECT

In [4]:

collective_stats = CollectiveStats()


seasons = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
playoff_dates = ['2016-04-13','2017-04-12','2018-04-11','2019-04-10',
                 '2020-08-14','2021-05-16','2022-04-10','2023-04-09',
                 '2024-04-14','2025-04-13']
critical_dates = {seasons[i]: playoff_dates[i] for i in range(len(seasons))}


for year in seasons:
    all_games = open_json(f'games/all_games_{year}.json')
    season = open_json(f'seasons/{year}_season.json')['response']
    game_dates = {str(game['id']): game['date']['start'] for game in season}

    for game_id, game in all_games.items():
        players_stats = game['response']


        regular_season = not compare_dates(game_dates[game_id], critical_dates[year])

        for p in players_stats:
            if p['player']['id'] is None or p['points'] is None: continue
            if p['comment'] and ('DNP' in p['comment'] or 'DND' in p['comment']): continue

            player_id = p['player']['id']
            team_id = p['team']['id'] if 'team' in p else None  


            game_stats = {key: p[key] for key in p if isinstance(p[key], (int, float))}


            for stat_name, stat_value in game_stats.items():
                collective_stats.add_league_stat(year, regular_season, game_id, stat_name, stat_value)
                if team_id:
                    collective_stats.add_team_stat(year, regular_season, game_id, str(team_id), stat_name, stat_value)


# POPULATE PLAYERS STATS WITH PERs

In [ ]:
#we are going to now go through every game, and every player who participated, and calculate their PERs for that game.

